In [0]:
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
from PIL import ImageEnhance
import glob
import gc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path_nv_others_nv_train = "/content/drive/My Drive/architecture one /nv_others/nv/train/"
path_nv_others_nv_validation = "/content/drive/My Drive/architecture one /nv_others/nv/validation/"
path_nv_others_others_train = "/content/drive/My Drive/architecture one /nv_others/others/train/"
path_nv_others_others_validation = "/content/drive/My Drive/architecture one /nv_others/others/validation/"
path_mel_others_mel_train = "/content/drive/My Drive/architecture one /mel_others/mel/train/"
path_mel_others_mel_validation= "/content/drive/My Drive/architecture one /mel_others/mel/validation/"
path_mel_others_others_train = "/content/drive/My Drive/architecture one /mel_others/others/train/"
path_mel_others_others_validation = "/content/drive/My Drive/architecture one /mel_others/others/validation/"
path_bkl_others_bkl_train= "/content/drive/My Drive/architecture one /bkl_others/bkl/train/"
path_bkl_others_bkl_validation="/content/drive/My Drive/architecture one /bkl_others/bkl/validation/"
path_bkl_others_others_train = "/content/drive/My Drive/architecture one /bkl_others/others/train/"
path_bkl_others_others_validation = "/content/drive/My Drive/architecture one /bkl_others/others/validation/"
path_bcc_akice_df_vasc_bcc_train = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/bcc/train/"
path_bcc_akice_df_vasc_bcc_validtion = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/bcc/validation/"
path_bcc_akice_df_vasc_akice_train = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/akice/train/"
path_bcc_akice_df_vasc_akice_validation  = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/akice/validation/"
path_bcc_akice_df_vasc_df_train = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/df/train/"
path_bcc_akice_df_vasc_df_validation = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/df/validation/"
path_bcc_akice_df_vasc_vasc_train = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/vasc/train/"
path_bcc_akice_df_vasc_vasc_validation = "/content/drive/My Drive/architecture one /bcc_akice_df_vasc/vasc/validation/"



In [0]:
from PIL import Image
import numpy as np
import os
import cv2
def noisy(image):
      row,col,ch = image.shape
      s_vs_p = 0.5
      amount = 0.03
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out


from PIL import ImageFilter
from PIL import Image

def rgb_to_hsv(rgb):
    # Translated from source of colorsys.rgb_to_hsv
    # r,g,b should be a numpy arrays with values between 0 and 255
    # rgb_to_hsv returns an array of floats between 0.0 and 1.0.
    rgb = rgb.astype('float')
    hsv = np.zeros_like(rgb)
    # in case an RGBA array was passed, just copy the A channel
    hsv[..., 3:] = rgb[..., 3:]
    r, g, b = rgb[..., 0], rgb[..., 1], rgb[..., 2]
    maxc = np.max(rgb[..., :3], axis=-1)
    minc = np.min(rgb[..., :3], axis=-1)
    hsv[..., 2] = maxc
    mask = maxc != minc
    hsv[mask, 1] = (maxc - minc)[mask] / maxc[mask]
    rc = np.zeros_like(r)
    gc = np.zeros_like(g)
    bc = np.zeros_like(b)
    rc[mask] = (maxc - r)[mask] / (maxc - minc)[mask]
    gc[mask] = (maxc - g)[mask] / (maxc - minc)[mask]
    bc[mask] = (maxc - b)[mask] / (maxc - minc)[mask]
    hsv[..., 0] = np.select(
        [r == maxc, g == maxc], [bc - gc, 2.0 + rc - bc], default=4.0 + gc - rc)
    hsv[..., 0] = (hsv[..., 0] / 6.0) % 1.0
    return hsv

def hsv_to_rgb(hsv):
    # Translated from source of colorsys.hsv_to_rgb
    # h,s should be a numpy arrays with values between 0.0 and 1.0
    # v should be a numpy array with values between 0.0 and 255.0
    # hsv_to_rgb returns an array of uints between 0 and 255.
    rgb = np.empty_like(hsv)
    rgb[..., 3:] = hsv[..., 3:]
    h, s, v = hsv[..., 0], hsv[..., 1], hsv[..., 2]
    i = (h * 6.0).astype('uint8')
    f = (h * 6.0) - i
    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))
    i = i % 6
    conditions = [s == 0.0, i == 1, i == 2, i == 3, i == 4, i == 5]
    rgb[..., 0] = np.select(conditions, [v, q, p, p, t, v], default=v)
    rgb[..., 1] = np.select(conditions, [v, v, v, q, p, p], default=t)
    rgb[..., 2] = np.select(conditions, [v, p, t, v, v, q], default=p)
    return rgb.astype('uint8')


def shift_hue(arr,hout):
    hsv=rgb_to_hsv(arr)
    hsv[...,0]=hout
    rgb=hsv_to_rgb(hsv)
    return rgb

#img = Image.open('m.m1.jpg').convert('RGBA')


#arr = np.array(img)
#blue_hue = 240/360.0
#new_img = (Image.fromarray(shift_hue(arr,blue_hue), 'RGBA')).convert('RGB')
#new_img.save('mm_blue.jpg')


In [0]:
def hue_trans(img,ids,save_path):
   img = img.convert('RGBA')
   arr = np.array(img)
   blue_hue = 240/360.0
   new_img = (Image.fromarray(shift_hue(arr,blue_hue), 'RGBA')).convert('RGB')
   new_img.save(save_path+ids+'_hue'+'.jpg')


this section below for training


In [0]:
#nv train
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/nv/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img.save(path_nv_others_nv_train+ids+'.jpg')
    img1.save(path_nv_others_nv_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_nv_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_nv_train)
    
    x=x+1
gc.enable()  
print(x)

5364


In [11]:
#bkl train / nv_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/bkl/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img3=img.transpose(Image.ROTATE_90)
    img4=img3.transpose(Image.FLIP_LEFT_RIGHT)
    img5=img3.transpose(Image.FLIP_TOP_BOTTOM)
    #img5=img3.transpose(Image.FLIP_TOP_BOTTOM)
    #bkl_train
    img.save(path_bkl_others_bkl_train+ids+".jpg")
    img1.save(path_bkl_others_bkl_train+ids+'_fv'+'.jpg')
    img2.save(path_bkl_others_bkl_train+ids+'_fh'+'.jpg')
    img3.save(path_bkl_others_bkl_train+ids+"_rot.jpg")
    img4.save(path_bkl_others_bkl_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_bkl_others_bkl_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_bkl_others_bkl_train)
    hue_trans(img3,ids,path_bkl_others_bkl_train)
    #others of nv_others
    img.save(path_nv_others_others_train+ids+".jpg")
    img1.save(path_nv_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_others_train+ids+'_fh'+'.jpg')
    img3.save(path_nv_others_others_train+ids+"_rot.jpg")
    img4.save(path_nv_others_others_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_nv_others_others_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_others_train)
    hue_trans(img3,ids,path_nv_others_others_train)
    x+=1
gc.enable()   
print(x)

879


In [12]:
#mel train / nv_others / bkl_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/mel/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img3=img.transpose(Image.ROTATE_90)
    img4=img3.transpose(Image.FLIP_LEFT_RIGHT)
    img5=img3.transpose(Image.FLIP_TOP_BOTTOM)

    #train
    #mel train
    img.save(path_mel_others_mel_train+ids+".jpg")
    img1.save(path_mel_others_mel_train+ids+'_fv'+'.jpg')
    img2.save(path_mel_others_mel_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_mel_others_mel_train)
    #others of nv_others
    img.save(path_nv_others_others_train+ids+".jpg")
    img1.save(path_nv_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_others_train+ids+'_fh'+'.jpg')
    img3.save(path_nv_others_others_train+ids+"_rot.jpg")
    img4.save(path_nv_others_others_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_nv_others_others_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_others_train)
    hue_trans(img3,ids,path_nv_others_others_train)
    #others of bkl_others
    img.save(path_bkl_others_others_train+ids+".jpg")
    img1.save(path_bkl_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_bkl_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bkl_others_others_train)
    x+=1
gc.enable()  
print(x)  

890


In [13]:
#bcc train /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/bcc/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img3=img.transpose(Image.ROTATE_90)
    img4=img3.transpose(Image.FLIP_LEFT_RIGHT)
    img5=img3.transpose(Image.FLIP_TOP_BOTTOM)

    #train
    #bcc train
    img.save(path_bcc_akice_df_vasc_bcc_train+ids+'.jpg')
    img1.save(path_bcc_akice_df_vasc_bcc_train+ids+'_fv'+'.jpg')
    img2.save(path_bcc_akice_df_vasc_bcc_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bcc_akice_df_vasc_bcc_train)
    #others of nv_others
    img.save(path_nv_others_others_train+ids+".jpg")
    img1.save(path_nv_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_others_train+ids+'_fh'+'.jpg')
    img3.save(path_nv_others_others_train+ids+"_rot.jpg")
    img4.save(path_nv_others_others_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_nv_others_others_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_others_train)
    hue_trans(img3,ids,path_nv_others_others_train)
    #others of bkl_others
    img.save(path_bkl_others_others_train+ids+".jpg")
    img1.save(path_bkl_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_bkl_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bkl_others_others_train)
    #others of mel_others
    img.save(path_mel_others_others_train+ids+".jpg")
    img1.save(path_mel_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_mel_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_mel_others_others_train)
    x+=1
gc.enable()  
print(x)

411


In [14]:
#akice train /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/akiec/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img3=img.transpose(Image.ROTATE_90)
    img4=img3.transpose(Image.FLIP_LEFT_RIGHT)
    img5=img3.transpose(Image.FLIP_TOP_BOTTOM)
     #train
    #akice train
    img.save(path_bcc_akice_df_vasc_akice_train+ids+'.jpg')
    img1.save(path_bcc_akice_df_vasc_akice_train+ids+'_fv'+'.jpg')
    img2.save(path_bcc_akice_df_vasc_akice_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bcc_akice_df_vasc_akice_train)
    #others of nv_others
    img.save(path_nv_others_others_train+ids+".jpg")
    img1.save(path_nv_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_others_train+ids+'_fh'+'.jpg')
    img3.save(path_nv_others_others_train+ids+"_rot.jpg")
    img4.save(path_nv_others_others_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_nv_others_others_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_others_train)
    hue_trans(img3,ids,path_nv_others_others_train)
    #others of bkl_others
    img.save(path_bkl_others_others_train+ids+".jpg")
    img1.save(path_bkl_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_bkl_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bkl_others_others_train)
    #others of mel_others
    img.save(path_mel_others_others_train+ids+".jpg")
    img1.save(path_mel_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_mel_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_mel_others_others_train)
    x+=1
gc.enable()    
print(x)

261


In [15]:
#vasc train /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/vasc/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img3=img.transpose(Image.ROTATE_90)
    img4=img3.transpose(Image.FLIP_LEFT_RIGHT)
    img5=img3.transpose(Image.FLIP_TOP_BOTTOM)
     #train
    #vasc train
    img.save(path_bcc_akice_df_vasc_vasc_train+ids+'.jpg')
    img1.save(path_bcc_akice_df_vasc_vasc_train+ids+'_fv'+'.jpg')
    img2.save(path_bcc_akice_df_vasc_vasc_train+ids+'_fh'+'.jpg')
    img3.save(path_bcc_akice_df_vasc_vasc_train+ids+"_rot.jpg")
    img4.save(path_bcc_akice_df_vasc_vasc_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_bcc_akice_df_vasc_vasc_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_bcc_akice_df_vasc_vasc_train)
    hue_trans(img3,ids,path_bcc_akice_df_vasc_vasc_train)
    #others of nv_others
    img.save(path_nv_others_others_train+ids+".jpg")
    img1.save(path_nv_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_others_train+ids+'_fh'+'.jpg')
    img3.save(path_nv_others_others_train+ids+"_rot.jpg")
    img4.save(path_nv_others_others_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_nv_others_others_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_others_train)
    hue_trans(img3,ids,path_nv_others_others_train)
    #others of bkl_others
    img.save(path_bkl_others_others_train+ids+".jpg")
    img1.save(path_bkl_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_bkl_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bkl_others_others_train)
    #others of mel_others
    img.save(path_mel_others_others_train+ids+".jpg")
    img1.save(path_mel_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_mel_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_mel_others_others_train)
    x+=1
gc.enable()    
print(x)

113


In [16]:
#df train /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/df/train/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img1=img.transpose(Image.FLIP_LEFT_RIGHT)
    img2=img.transpose(Image.FLIP_TOP_BOTTOM)
    img3=img.transpose(Image.ROTATE_90)
    img4=img3.transpose(Image.FLIP_LEFT_RIGHT)
    img5=img3.transpose(Image.FLIP_TOP_BOTTOM)
     #train
    #vasc train
    img.save(path_bcc_akice_df_vasc_df_train+ids+'.jpg')
    img1.save(path_bcc_akice_df_vasc_df_train+ids+'_fv'+'.jpg')
    img2.save(path_bcc_akice_df_vasc_df_train+ids+'_fh'+'.jpg')
    img3.save(path_bcc_akice_df_vasc_df_train+ids+"_rot.jpg")
    img4.save(path_bcc_akice_df_vasc_df_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_bcc_akice_df_vasc_df_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_bcc_akice_df_vasc_df_train)
    hue_trans(img3,ids,path_bcc_akice_df_vasc_df_train)
    #others of nv_others
    img.save(path_nv_others_others_train+ids+".jpg")
    img1.save(path_nv_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_nv_others_others_train+ids+'_fh'+'.jpg')
    img3.save(path_nv_others_others_train+ids+"_rot.jpg")
    img4.save(path_nv_others_others_train+ids+'_rot_fv'+'.jpg')
    img5.save(path_nv_others_others_train+ids+'_rot_fh'+'.jpg')
    hue_trans(img,ids,path_nv_others_others_train)
    hue_trans(img3,ids,path_nv_others_others_train)
    #others of bkl_others
    img.save(path_bkl_others_others_train+ids+".jpg")
    img1.save(path_bkl_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_bkl_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_bkl_others_others_train)
    #others of mel_others
    img.save(path_mel_others_others_train+ids+".jpg")
    img1.save(path_mel_others_others_train+ids+'_fv'+'.jpg')
    img2.save(path_mel_others_others_train+ids+'_fh'+'.jpg')
    hue_trans(img,ids,path_mel_others_others_train)
    x+=1
gc.enable()    
print(x)

92


section below for validation

In [17]:
#nv validation
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/nv/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    img.save(path_nv_others_nv_validation+ids+'.jpg')
    x=x+1
gc.enable()  
print(x)


1341


In [18]:
#bkl validation / nv_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/bkl/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    
    #img5=img3.transpose(Image.FLIP_TOP_BOTTOM)
    #bkl_validation
    img.save(path_bkl_others_bkl_validation+ids+".jpg")
    #others of nv_others
    img.save(path_nv_others_others_validation+ids+".jpg")
    
    x+=1
gc.enable()   
print(x)

220


In [19]:
#mel validation / nv_others / bkl_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/mel/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)

    #validation
    #mel validation
    img.save(path_mel_others_mel_validation+ids+".jpg")
    
    #others of nv_others
    img.save(path_nv_others_others_validation+ids+".jpg")
    
    #others of bkl_others
    img.save(path_bkl_others_others_validation+ids+".jpg")
    
    x+=1
gc.enable()  
print(x)  

223


In [21]:
#bcc validation /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/bcc/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    

    #validation
    #bcc validation
    img.save(path_bcc_akice_df_vasc_bcc_validtion+ids+'.jpg')
    
    #others of nv_others
    img.save(path_nv_others_others_validation+ids+".jpg")
    
    #others of bkl_others
    img.save(path_bkl_others_others_validation+ids+".jpg")
    
    #others of mel_others
    img.save(path_mel_others_others_validation+ids+".jpg")
    
    x+=1
gc.enable()  
print(x)

103


In [25]:
#akice validation /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/akiec/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    
     #validation
    #akice validation
    img.save(path_bcc_akice_df_vasc_akice_validation+ids+'.jpg')
    
    #others of nv_others
    img.save(path_nv_others_others_validation+ids+".jpg")
    
    #others of bkl_others
    img.save(path_bkl_others_others_validation+ids+".jpg")
    
    #others of mel_others
    img.save(path_mel_others_others_validation+ids+".jpg")
    
    x+=1
gc.enable()    
print(x)

66


In [26]:
#vasc validation /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/vasc/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    
     #validation
    #vasc validation
    img.save(path_bcc_akice_df_vasc_vasc_validation+ids+'.jpg')
    
    #others of nv_others
    img.save(path_nv_others_others_validation+ids+".jpg")
    
    #others of bkl_others
    img.save(path_bkl_others_others_validation+ids+".jpg")
    
    #others of mel_others
    img.save(path_mel_others_others_validation+ids+".jpg")
    
    x+=1
gc.enable()    
print(x)

29


In [27]:
#df validation /nv_others/bkl_others/mel_others
gc.disable()
x=0
for filename in glob.glob("/content/drive/My Drive/color_cons_skin_cancer_crop/df/validation/*.jpg"): #assuming gif    
    path = filename.split("/")
    name = path[7]  
    parts=name.split(".")
    ids=parts[0]                          
    img=Image.open(filename)
    
     #validation
    #vasc validation
    img.save(path_bcc_akice_df_vasc_df_validation+ids+'.jpg')
    
    #others of nv_others
    img.save(path_nv_others_others_validation+ids+".jpg")
    
    #others of bkl_others
    img.save(path_bkl_others_others_validation+ids+".jpg")
    
    #others of mel_others
    img.save(path_mel_others_others_validation+ids+".jpg")
    
    x+=1
gc.enable()    
print(x)

23
